## Query Processing Module

#### Query Input

In [2]:
import nltk
# nltk.download('stopwords')
from pyvis.network import Network
import networkx as nx
import pickle
import bs4

%matplotlib inline

DATA="../data/"

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# from os import listdir
# from os.path import isfile, join
# mypath = "text_misc_clean/"
# misc_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# mypath = "text_casefiles/"
# case_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]


1.15.0


In [4]:
from nltk.metrics import edit_distance
#Definitions dictionary
import os, json
import pickle
defdict = {}
path_to_json = DATA + 'Definitions/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for i in json_files:
    with open(DATA + 'Definitions/'+i) as json_file:
        data = json.load(json_file)
    defdict.update(data)
definitions = list(defdict.keys())
defvalues = list(defdict.values())

docs = ["Issue and Listing of Non Convertible Redeemable Preference Shares", "Investment Advisers", "Depositories and Participants", "Mutual Funds", "Employees Service", "Substantial Acquisition of Shares and Takeovers", "Appointment of Administrator and Procedure for Refunding to the Investors", "Prohibition of Fraudulent and Unfair Trade Practices relating to Securities Market", "Know Your Client Regulations", "Prohibition of Insider Trading", "Merchant Bankers", "Issue and Listing  of Securities Debt Instruments and Security Receipts", "Delisting of Equity Shares","Issue of Capital And Disclosure Requirements2", "Foreign Venture Capital Investor", "Procedure for Board Meetings", "Custodian", "Ombudsman", "Investor Protection and Education Fund", "Foreign Portfolio Investors", "Issue of Sweat Equity", "Collective Investment Scheme", "Portfolio Managers", "Research Analysts", "Procedure for Search and Seizure", "Issue of Capital And Disclosure Requirements", "Share Based Employee Benefits", "Debenture Trustees", "Alternative Investment Funds", "Stock Exchanges and Clearing Corporations", "Self Regulatory Organisations", "Settlement Proceedings", "Issues and Listing of Muncipal Debt Securities", "Buy Back Of Securities2","Issue and Listing of Debt Securities", "Infrastructure Investment Trusts", "Stock Brokers", "Listing Obligations and Disclosure Requirements", "Registrars to an Issue and Share Transfer Agents", "Real Estate Investment Trusts", "Intermediaries", "Certification of Associated Persons in the Securities Markets", "Credit Rating Agencies", "Regulatory Fee on Stock Exchanges", "Underwriters", "Buy Back Of Securities", "Bankers to an Issue", "Central Database of Market Participants"]

#Regulations
finalclean48 = []
with open(DATA + 'cleanedregulations48.pkl','rb') as f:
    finalclean48 = pickle.load(f)    
docregs = finalclean48

#Topics documentwise
finalcleantopics48 = []
with open(DATA + 'cleanedregtopics48.pkl','rb') as f:
    finalcleantopics48 = pickle.load(f)    
finaltopics = finalcleantopics48

#vocab definitions
mainvocab = []
with open(DATA + 'mainvocab.pkl','rb') as f:
    mainvocab = pickle.load(f) 

vocabdef = []
with open(DATA + 'vocabdef.pkl','rb') as f:
    vocabdef = pickle.load(f) 

#Additional Documents

with open(DATA + 'concept_filenames.pkl','rb') as f:
    cfile = pickle.load(f) 
with open(DATA + 'concept_sentences.pkl','rb') as f:
    csent = pickle.load(f) 
with open(DATA + 'concept_text.pkl','rb') as f:
    ctext = pickle.load(f) 

with open(DATA + 'informal_filenames.pkl','rb') as f:
    ifile = pickle.load(f) 
with open(DATA + 'informal_sentences.pkl','rb') as f:
    isent = pickle.load(f) 
with open(DATA + 'informal_text.pkl','rb') as f:
    itext = pickle.load(f) 
    
with open(DATA + 'miscvocab.pkl', 'rb') as f:
    miscvocab = pickle.load(f)
    
#Legal Case files
with open(DATA + 'case_filenames.pkl','rb') as f:
    lfile = pickle.load(f) 
with open(DATA + 'case_sentences.pkl','rb') as f:
    lsent = pickle.load(f) 
with open(DATA + 'case_text.pkl','rb') as f:
    ltext = pickle.load(f) 
    
with open(DATA + 'casevocab.pkl', 'rb') as f:
    casevocab = pickle.load(f)

import spacy
nlp = spacy.load('en_core_web_sm')

from nltk.corpus import wordnet

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import math
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
STOPWORDS = set(
    list(stop_words.ENGLISH_STOP_WORDS) +\
    stopwords.words('english') +\
    ['mm', 'section', 'subsection', 'schedule', '-PRON-', 'chapter', 'regulation', 'repealed', 'thereto','unpublishe', 'thereunder','guideline', 'reference','onus','make','Page','Securities','Exchange','India'])

# from cdqa.pipeline import QAPipeline

# from cdqa.utils.download import download_model
# download_model(model='bert-squad_1.1', dir='./models')

with open(DATA + 'glossary.json') as f:
    glossary = json.load(f)

In [5]:
def queryvocab(query):
    query1 = ""
    qvocab=[]
    for i in query.split():
        if(i not in stopwords.words()):
            if(i != 'What' and i != 'what' and i!='How'):
                qvocab.append(i)
                query1 = query1 + " " + i
    query = query1
    return qvocab, query

In [6]:
def regextract(docinput, docs, docregs, mainvocab, vocabdef, query, glossary):
    
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    v = []
    
    if(docinput == 'All'):
        rdocs = []
        for i in glossary.keys():
            if(i in query.lower()):
                dval = glossary[i]
                for j in dval:
                    if(j not in rdocs):
                        rdocs.append(j)
        if(len(rdocs)>0):
            docinput = []
            for i in rdocs:
                docinput.append(docs[i])
        else:
            docinput = ['All']
            
    else:
        d = [docinput]
        docinput = d
    
    for dval in docinput:
    
        if(dval != 'All'):
            ind = docs.index(dval)

            rules = docregs[ind]
        
            t=-1
            vocab0 = []
            for i in rules:
                t=t+1
                vocab0.append(i.split())
            
            for i in vocab0:
                for j in i:
                    if(len(j)>2):
                        for m in range(len(symbols)):
                            j = np.char.replace(j, symbols[m], '')
                            j = np.char.replace(j, ",", "")
                            j = np.array2string(j)
                            j = j.replace("'","")
                        if(j not in v):
                            v.append(j)
        else:
            rules = []
            for i in docregs:
                for j in i:
                    rules.append(j)
            vocab0 = mainvocab
        
            for j in vocab0:
                if(len(j)>2):
                    for m in range(len(symbols)):
                        j = np.char.replace(j, symbols[m], '')
                        j = np.char.replace(j, ",", "")
                        j = np.array2string(j)
                        j = j.replace("'","")
                    if(j not in v):
                        v.append(j)
    
    if('All' not in docinput):
        vdef = []
        for i in v:
            if(i in mainvocab and len(i)>2):
                vindex = mainvocab.index(i)
                vdef.append(vocabdef[vindex])
    else:
        vdef = vocabdef
        
    
    return rules, v, vdef, docinput

In [7]:
def querypreprocess(query, qvocab, definitions, finaltopics):
    if('regulations' in qvocab):
        qvocab.remove('regulations')
    if('rules' in qvocab):
        qvocab.remove('rules')
    if('rule' in qvocab):
        qvocab.remove('rule')
    if('chapter' in qvocab):
        qvocab.remove('chapter')
    if('section' in qvocab):
        qvocab.remove('section')
    if('sub' in qvocab):
        qvocab.remove('sub')
    if('SEBI' in qvocab):
        qvocab.remove('SEBI')
    if('means' in qvocab):
        qvocab.remove('means')
    if('shall' in qvocab):
        qvocab.remove('shall')
    if('Securities' in qvocab):
        qvocab.remove('Securities')
    if('Exchange' in qvocab):
        qvocab.remove('Exchange')
    if('pertaining' in qvocab):
        qvocab.remove('pertaining')
    if('India' in qvocab):
        qvocab.remove('India')
        
    query = ""
    for i in qvocab:
        query = query + " " + i
        
    importantwords = []
    for i in qvocab:
        flag = 0
        for j in definitions:
            if(i in j):
                flag = 1
                break
        if(flag == 1):
            importantwords.append(i)

    expansionwords = []
    for i in qvocab:
        flag=0
        for j in finaltopics:
            for k in j:
                if(i == k):
                    flag=1
        if(flag==1):
            if(i not in importantwords):
                importantwords.append(i)
        if(flag==0):
            for j0 in definitions:
                if(i in j0):
                    expansionwords.append(j0)
                    break
    
    qnew = query
    for j in expansionwords:
        k=0
        for i in definitions:
            if(i==j):
                s = i
                s = s + defvalues[k]
                qnew = qnew + ' ' + s
                break
            k=k+1
        else:
            qnew = qnew + ' ' + j
    query = qnew
    
    sent = nlp(query)
    t=0
    for token in sent:
        if(str(token) in importantwords):
            if(t!=0):
                if(str(sent[t-1]) not in importantwords):
                    importantwords.append(str(sent[t-1]))
            if(token.tag_ == 'VB'):
                importantwords.append(str(token))
        t+=1
    
    return query, qvocab, importantwords, expansionwords

## Regulations Retrieval Module

In [8]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = STOPWORDS
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [9]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = stemming(data)
    return data

In [10]:
def tfidfreg(docregs, docinput, category):
    
    if(category == 'regulations'):
        d = [] 
    
        for i in docinput:
            if(i in docs):
                ind = docs.index(i)
                r0 = docregs[ind]
                d.append(r0)
            
        if(len(d)>0):
            docregs = d

    regstring = []
    reverseMap = dict()
    origregstring = [] # flattened docregs
    for ind,i in enumerate(docregs):
        for j in i:
            origregstring.append(j)
            reverseMap[j]=docs[ind]
            data = preprocess(j)
            regstring.append(data)       
    
    
    wordregs = regstring
    
    t=0
    remind = []
    for i in regstring:
        if(i=='\xa0\n' or i == ''):
            remind.append(t)
            regstring.remove(i)
            wordregs.remove(wordregs[t])
        t+=1
        
    origreg = []
    t=0
    while(t<len(origregstring)):
        if(t not in remind):
            origreg.append(origregstring[t])
        t+=1
            
    wordregs = regstring
    
    DF = {}
    wlist = []
    for i in range(len(wordregs)):
        tokens = regstring[i]
        for w in tokens.split():
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
                wlist.append(w)
                
    for i in DF:
        DF[i] = len(DF[i])
    
    total_vocab = [x for x in DF]
    total_vocab_size = len(DF)

# TFIDF Calculation here
    tf_idf = {}
    N = len(regstring)
    for i in range(N):
        tokens = regstring[i].split()
        counter = Counter(tokens)
        for token in np.unique(tokens):
            if(token in wlist):
                tf = counter[token]/len(tokens)
                df = DF[token]
                idf = np.log((N+1)/(df+1))
                try:
                    tf_idf[token] = tf_idf[token] + tf*idf
                except:
                    tf_idf[token] = tf*idf                
            
    return DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg,reverseMap

In [11]:
def q_vector(qvocab, N, total_vocab, DF):
    Q = np.zeros((len(total_vocab)))
    querytokens = qvocab
    counter = Counter(querytokens)
    words_count = len(querytokens)

    query_weights = {}
    
    for token in np.unique(querytokens):
        if(token in DF):
            tf = counter[token]/words_count
            df = DF[token]
            idf = math.log((N+1)/(df+1))

            try:
                ind = total_vocab.index(token)
                Q[ind] = tf*idf
            except:
                pass
    return Q

In [12]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [13]:
def cosine_similarity(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords,reverseMap):
    d_cosines = []
    
    query_vector = q_vector(qvocab, N, total_vocab, DF)

    t=0
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        t=t+1
        
    out = np.array(d_cosines).argsort()[::-1]
    answers = []
    indexs=[]
    for i in out:
        flag=0
        for j in qvocab:
            if(j in importantwords):
                flag=flag+2
            else:
                flag=flag+1
        if(flag>1):
            answers.append([flag,origreg[i]])

    answers.sort(reverse=True)
    distances=list()
    for ele in answers:
        distances.append(ele[0])
        if ele[1] in reverseMap.keys():
            indexs.append(reverseMap[ele[1]])
        else:
            indexs.append(None)
        
    if(k<len(answers)):
        return answers[0:k],indexs,distances
    else:
        return answers,indexs,distances


In [14]:
from sentence_transformers import SentenceTransformer
from pdb import set_trace
# ;set_trace()
def cosine_similarity_transformer(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords,reverseMap):
    
#     find between query and each origreg 

    d_cosines = []
#     model_name = 'bert-base-nli-mean-tokens'
    model_name='sentence-transformers/roberta-base-nli-stsb-mean-tokens'
    model = SentenceTransformer(model_name)
    query_vector = model.encode(query)
    reg_vectors= model.encode(origreg)
#     from pdb import set_trace;
#     set_trace()    
    t=0
    for r in reg_vectors:
        d_cosines.append(cosine_sim(query_vector, r))
        t=t+1
        
    out = np.array(d_cosines).argsort()[::-1]
    answers = []
    indexs=[]
    for i in out:
        flag=0
        for j in qvocab:
            if(j in importantwords):
                flag=flag+2
            else:
                flag=flag+1
        if(flag>1):
            answers.append([flag,origreg[i]])

    answers.sort(reverse=True)
    distances=list()
    for ele in answers:
        distances.append(ele[0])
        if ele[1] in reverseMap.keys():
            indexs.append(reverseMap[ele[1]])
        else:
            indexs.append(None)
        
    if(k<len(answers)):
        return answers[0:k],indexs,distances
    else:
        return answers,indexs,distances


In [15]:
def vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg,importantwords,reverseMap):
    D = np.zeros((N, len(total_vocab)))
    for i in tf_idf:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf[i]
        except:
            pass
    A,indexs,distances = cosine_similarity_transformer(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords,reverseMap)
    
    return A,indexs,distances

In [16]:
# def answermod(A, queryinput):
    
#     ans = []
#     ansreg = []
    
#     t=0
#     while(t<len(A)):
#         A[t][0] = 'title'
#         A[t][1] = [A[t][1]]
#         t+=1
#     t=0
#     df = pd.DataFrame(A,columns=['title','paragraphs'])
    
#     cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
#     cdqa_pipeline.fit_retriever(df=df)
#     print("Predicting")
#     prediction = cdqa_pipeline.predict(queryinput)
#     print("Finished prediction")
#     return prediction[0], prediction[2]

In [17]:
from transformers import pipeline
import json
def answermod(A,queryinput):
    model_name = "deepset/roberta-base-squad2" # roberta QnA model
    model = pipeline(model=model_name, tokenizer=model_name, revision="v1.0", task="question-answering")
    answers=list()
    scores=list()
    regs = [a[1] for a in A]
    for r in regs:
        try:
            result=model(question = queryinput, context= r)
            answers.append(result["answer"])
            scores.append(result["score"])
        except:
            print(r)
            print("=================")
    out = np.array(scores).argsort()[::-1]
    return answers[out[0]], regs[out[0]] , scores[out[0]] # Returns the high score answer,context and the score

In [18]:
from transformers import AutoModel, AutoTokenizer
import torch
# LEGAL bert does not have a question answering version, unable to make use of it
def answermod(A,queryinput):
    model_name = "nlpaueb/legal-bert-base-uncased" # Legal Bert
    model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    regs = [a[1] for a in A]
    answers=list()
    scores=list()
    nlp = pipeline(model=model, tokenizer=tokenizer, task="question-answering")    
    for r in regs:
#         try:
        result=nlp(question = queryinput, context= r)
        answers.append(result["answer"])
        scores.append(result["score"])
#         except:
#             print(r)
#             print("=================")
    
    out = np.array(scores).argsort()[::-1]
    return answers[out[0]], regs[out[0]] , scores[out[0]] # Returns the high score answer,context and the score

In [19]:
def QnAmodel(queryinput, docinput,k,category):
    print('Preprocessing query......')
    qvocab, query = queryvocab(queryinput)
    if(category == 'regulations'):
        rules, vocab, vdef, docinput = regextract(docinput, docs, docregs, mainvocab, vocabdef, queryinput, glossary)
        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('tf-idf calculation in progress.....')
        DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg,reverseMap = tfidfreg(docregs,docinput,category)
        return reverseMap
#  Can uncomment below lines to save dump
#         with open(DATA + 'tfidf_regs.pkl','wb') as f:
#             pickle.dump(tf_idf,f)
        print('Extracting relevant answer regulations.......')
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A
    elif(category == 'misc'):
        
        rules = ctext
        rules.extend(itext)
        
        vocab = miscvocab
        N = 440 
        
        with open(DATA + 'misc_tf_idf_new.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open(DATA + 'misc_total_vocab_new.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open(DATA + 'misc_DF_new.json') as f:
            DF = json.load(f)
        with open(DATA + 'misc_origreg_new.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        with open(DATA + 'miscfiles_mapper.pkl','rb') as filer:
            reverseMap = pickle.load(filer)

        origreg=rules
        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
        
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A
        
        sentences = csent
        sentences.extend(isent)
        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
    elif(category == 'legal case'):
        rules = ltext
        vocab = casevocab
        N = 1496 
        
        with open(DATA + 'case_tf_idf_new.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open(DATA + 'case_total_vocab_new.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open(DATA + 'case_DF_new.json') as f:
            DF = json.load(f)
        with open(DATA + 'case_origreg_new.pkl', 'rb') as f:
            origreg = pickle.load(f)
            
        with open(DATA + 'casefiles_mapper.pkl','rb') as filer:
            reverseMap = pickle.load(filer)

        with open(DATA + 'casefiles_sentences_new.pkl','rb') as filer:
            case_sentences = pickle.load(filer)

        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
                
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A

        Anew = []
        for ans in A:
            ind = origreg.index(ans[1])
            for sent in case_sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
        
    else:
        return None
        
    print('Extracting answer')
    answer, ansreg ,confidence_score= answermod(A, queryinput)
    print("The confidence score in answer is:\n",confidence_score)
    return answer, ansreg, Aold,indexs,distances
#     return None, None, Aold,indexs,distances
#     return Aold


## User input

In [20]:
with open(DATA + 'casefiles_mapper.pkl','rb') as filer:
    caseMap = pickle.load(filer)
with open(DATA + 'miscfiles_mapper.pkl','rb') as filer:
    miscMap = pickle.load(filer)



In [60]:
regMap = dict()
for ind,i in enumerate(docregs):
    for j in i:
        regMap[j]=docs[ind]


In [23]:
# queryinput = "What are the legal  provisions  on  appointment  and  re-appointment  of directors  of  a listed entity?"
# docinput = 'All'
# category = 'regulations'

# queryinput = "I am a shareholder in xyz company with my total share capital being 7%. I sold a part of my share percent which reduced my holding by 3%. Will I be required to disclose about the same to under Form C to the company since my total shareholding has fallen below 5%?"
# docinput = 'All'
# category = 'legal case'

queryinput = "How much amount will be provided as reward for informants in insider trading cases"
docinput = 'All'
category = 'regulations'

# queryinput = "What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?"
# docinput = 'All'
# category = 'misc'

k=5
# answer,ansreg,Aold,indexs,distances = QnAmodel(queryinput,docinput,k, category)
regMap = QnAmodel(queryinput,docinput,k, category)

Preprocessing query......
tf-idf calculation in progress.....


In [39]:
print(queryinput)
print()
print(answer)
print()
print(ansreg)

What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?

acquisition of shares, voting rights or control

MassMutual, Baring and Oppenheimer do not have any formal or informal agreements or understandings to co-operate in the acquisition of shares, voting rights or control of Indian-listed issuers.


In [40]:
Anew=list()
for x in Aold:
    Anew.append(x[1])
Anew = Anew[:k]
docnames=indexs[:k]
dist=distances[:k]
targets = [i for i in range(len(docnames))]

In [42]:
def resultgraph(targets,answers,doclinks,queryinput,distances):
    idoc="results"
    docnumber = 2
    
    q = 'Results map '
    edge_data = zip(targets, answers,doclinks,distances)
    
    nt = Network(height="100%", width="100%",heading=q)
    nt.hrepulsion()
        
    nt.add_node("questionDot", label='question', title=queryinput, color = "red")
        
    t=0
    igcount=0
    lccount=0
    for e in edge_data:
        dst = e[0]
        h = e[1]
        pdflink = e[2]
        linker = pdflink+'.pdf'
        if pdflink+'.docx' in ifile:
            linker = pdflink+'.docx'
        nt.add_node(str(dst), label=pdflink, title=h, color = "blue", url='./PDFS/'+linker)
        
        nt.add_edge("questionDot", str(dst), value=8, color = "#000000", length=e[3])
        
        t=t+1
    
    nt.show_buttons()  # Allows for adjustments in UI
    nt.show(idoc+".html") # saves it, check for show so that it is automatically opened for user
    
    with open(idoc+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]

    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)

    t=0
    appendind = []
    for i in tokens1:
        if('return' in i):
            appendind.append(t)
        t=t+1
    
    with open("interactive.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<appendind[-1]):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = appendind[-1]
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s

    soup.find_all('script',type="text/javascript")[-1] = tag
    with open(idoc+'.html', "w") as outf:
        outf.write(str(soup))
        
    with open(idoc+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]
    
    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)
    
    t0=0
    while(t0<len(tokens1)):
        if('"edges": {' in tokens1[t0]):
            break
        t0+=1
        
    with open("nodefont.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<t0):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = t0
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s
    
    soup.find_all('script',type="text/javascript")[-1] = tag
    with open(idoc+'.html', "w") as outf:
        outf.write(str(soup))


In [43]:
resultgraph(targets,Anew,docnames,queryinput,dist)

### Example outputs from model

In [24]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


How much amount will be provided as reward for informants in insider trading cases


The amount of the Reward, if payable


7F.  (1)  Informants  who  are  considered  tentatively  eligible  for  a  Reward,  shall  submit  the Informant Reward Claim Form set out in Schedule E to the Board within the period specified in the intimation sent by the Board.


In [25]:
queryinput = "As of date, what are the provisions for a person to get appointed of persons who fail to get elected as Whole time Directors /Managing Directors at the general meeting of a listed entity?"              
docinput = 'All'
category = 'misc'
answer, ansreg, Aold = QnAmodel(queryinput,docinput,10, category)

Preprocessing query......
Extracting relevant answer regulations.......


/home/spawnfire/anaconda3/envs/testingEnvironment/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


Extracting answer











The confidence score in answer is:
 0.08061201870441437


ValueError: too many values to unpack (expected 3)

In [ ]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


In [ ]:
queryinput = "What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?"
docinput = 'All'
category = 'misc'
answer, ansreg, Aold = QnAmodel(queryinput,docinput,10, category)

In [ ]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


In [ ]:
queries_case = ["I am a shareholder in xyz company with my total share capital being 7%. I sold a part of my share percent which reduced my holding by 3%. Will I be required to disclose about the same to under Form C to the company since my total shareholding has fallen below 5%?"
               ,"I am a shareholder in abc company with a shareholding of 10%. I recently acquired a few more shares which has increased my shareholding by 5%, bringing it to a total of 15%. What all disclosures do I need to make?"
               ,"PQR, a corporate entity has purchased 8986 shares and sold 928600 shares of ASQ.  PQR also purchased 9,86,094 shares from off-market. As of 31st December, 2004, PQR was holding 9,86,094 shares which was which was 9.26% of the share capital of ASQ. By March 31st, its shareholding was reduced to 0. What all violations have been made?"
               ,"GHI is a corporate entity that is a promoter of SPZ. As on March 2005, it held 2,30,59,317 shares constituting of 30.87% of the total shares. By the end of September 2005, its shareholding reduced by 1,41,92,187 to 19% . What all violations have been made?"
               ,"TPR is a company that is listed on both BSE & NSE. STC is a promoter of TPR. TPR took out Rights Issue and through that allotted 16,49,653 partly paid-up shares to STC on 2nd October, 2012. Consequently, the total shares of STC  increased from 1,03,18,427 constituting to 4.98% of the share capital to 1,19,68,080 shares constituting 5.06% of the total share capital of TPR. STC was informed about the allotment on 10th October, 2012. However, STC did not make disclosures about its increased shareholding under 13(1) of PIT Regulations and believed that since it was already a promoter of TPR, it instead made disclosure under 14(4A) of PIT Regulations. Accordingly it made disclosure under 13(4A) of PIT Regulations under Form D with a delay of 1 month and not under 13(1) of PIT Regulations under Form A. Is there any violation?"
               ,"Mr. X is a promoter and managing director of GTF Pharmaceuticals. He has pledged certain shares of the company and the pledge on these shares was subsequently released. Mr. X had created a pledge 1,20,000 shares on 11.12.2012 and release of pledge of 5,00,000 shares had taken place on 27.12.2012. Later it was observed that shareholding of Mr. X was changed after some off-market transactions. Mr. X has contended that 1,20,000 shares were pledged with YZ Financial on 11.12.2012 and the said transaction was intimated to stock exchanges on 21.12.2012 and there was a delay of 3 days and the same may be condoned. On 27.12.2012 the pledge on 5 lakh shares were released by DTR and the same was duly intimated to stock exchanges on 04.01.2013. With respect to the off-market transaction, Mr. X has contended that the same was No an off-market transaction but was only a transfer from the his one DP account to its another DP account. It has been alleged that Mr. X did not make the requisite disclosures under Regulation 31(1) &31(2) read with 31(3) of the SAST Regulations, 2011 and Regulation 13(4) & (4A) read with 13(5) SEBI (Prohibition of Insider Trading) Regulations,1992 (PIT Regulations). Has the violation occurred?"
               ,"Mr. X is a promoter and managing director of GTF Pharmaceuticals. He has pledged certain shares of the company and the pledge on these shares was subsequently released. Mr. X had created a pledge 1,20,000 shares on 11.12.2012 and release of pledge of 5,00,000 shares had taken place on 27.12.2012. Later it was observed that shareholding of Mr. X was changed after some off-market transactions. Mr. X has contended that 1,20,000 shares were pledged with YZ Financial on 11.12.2012 and the said transaction was intimated to stock exchanges on 21.12.2012 and there was a delay of 3 days and the same may be condoned. On 27.12.2012 the pledge on 5 lakh shares were released by DTR and the same was duly intimated to stock exchanges on 04.01.2013. With respect to the off-market transaction, Mr. X has contended that the same was No an off-market transaction but was only a transfer from the his one DP account to its another DP account. It has been alleged that Mr. X did not make the requisite disclosures under Regulation 31(1) &31(2) read with 31(3) of the SAST Regulations, 2011 and Regulation 13(4) & (4A) read with 13(5) SEBI (Prohibition of Insider Trading) Regulations,1992 (PIT Regulations). How much penalty should be imposed?"]

In [ ]:
for ind, queryinput in enumerate(queries_case):
    get_res(queryinput, 'All', 'legal case', "CaseFile_"+str(ind))

In [ ]:
queries_new_micro = ["What are some of the activities that being outsourced by intermediaries in the securities market?"
                    ,"Under what conditions can approval from shareholders be dispensed with for reclassification of promoter to public?"
                    ,"What is transmission of shares?"
                    ,"What are the provisions for transfer of shares?"]

In [ ]:
for ind, queryinput in enumerate(queries_new_micro):
    get_res(queryinput, 'All', 'misc', "New_Microsoft_doc_"+str(ind))

In [ ]:
queries_pit = ["Is it insider trading if I know an employee of a company and I buy that company’s stock?"
              ,"How much amount will be provided as reward for informants in insider trading cases by Securities and Exchange Board of India (SEBI)?"
              ,"How do CEOs and their families trade stock from their company without it being insider trading?"
              ,"When do CEOs and their families trade stock from their company without it being insider trading?"
              ,"What should be done by a company when it suspects insider trading by an employee?"
              ,"Who is a compliance officer and what is its role in preventing Insider trading?"]

In [ ]:
for ind, queryinput in enumerate(queries_pit):
    get_res(queryinput, 'All', 'regulations', "PIT_"+str(ind))

In [ ]:
queries_query_testing = ["What if a stock delisted from the market?"
                        ,"What can a person do with shares that are now delisted?"
                        ,"Why companies are delisting their shares?"
                        ,"If a company is about to delist and is ready to buy back my shares and I didn't sell, what will happen?"
                        ,"How does an initial public offering get valued?"
                        ,"How does a company offer initial public offering?"
                        ]

In [ ]:
for ind, queryinput in enumerate(queries_query_testing):
    get_res(queryinput, 'All', 'regulations', "query_testing_"+str(ind))

In [ ]:
queries_sast = ["What are the threshold limits for acquisition of shares / voting rights, beyond which an obligation to   make an open offer is triggered?"
               ,"What is a voluntary open offer?"
               ,"What are the restrictions on acquirers making a voluntary open offer?"]

In [ ]:
for ind, queryinput in enumerate(queries_sast):
    get_res(queryinput, 'All', 'regulations', "sast_"+str(ind))

In [ ]:
queries_sebi_consult = ["What are the legal  provisions  on  appointment  and  re-appointment  of directors  of  a listed entity?"
                       ,"As of date, what are the provisions for a person to get appointed of persons who fail to get elected as Whole time Directors /Managing Directors at the general meeting of a listed entity?"
                       ,"What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?"
                       ,"What is meant by 'Migration to Main Board'?"
                       ,"Why are Broad Market Indices created?"
                       ,"Which are the major Domestic & International broad market indices?"]

In [ ]:
for ind, queryinput in enumerate(queries_sebi_consult):
    get_res(queryinput, 'All', 'misc', "sebi_consult_"+str(ind))

In [ ]:
# Finalising on the finetuning code

In [ ]:
train_contexts, train_questions, train_answers = read_data('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_data('squad/dev-v2.0.json')


In [2]:
!mkdir squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json


--2021-10-15 15:37:32--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M  11.8MB/s    in 3.7s    

2021-10-15 15:37:39 (11.0 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2021-10-15 15:37:39--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

sq

In [3]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')


In [4]:
train_contexts[0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [5]:
train_questions[0]

'When did Beyonce start becoming popular?'

In [ ]:
# so we need context spans for each of these answer spans. 
# So for each of the answers
# Find the origreg parts that have the answer span
# manually create them as context
# after that is done can fit into the pipeline for finetuning
# 


# For the key words, for each file type
# go through origreg, as per number of times the given keywords are present
# thats the score for that element
# need to maintain map of score to document name

In [2]:
import pandas as pd
# train_answers[0]

In [3]:
df = pd.read_csv("QnA.csv")
df

,Question,Answer Span,Document,Keywords,File Type,Files For Keywords
0,Whether any entity that qualifies as a promote...,"With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e,"pre-clearance, compliance officer, designated ...",misc,"['7db3e9ce43373e8d2823b7603b72252b', '853914f4..."
1,Whether inter-se off-market transfer of shares...,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5,"inter-se off market transfer, block deal, cont...",misc,"['08137e72ee44f3d2f24e42b2ca270de6', '830be2cf..."
2,Whether an Alternative Investment Fund under t...,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639,"unutilized funds, Alternate Investment Funds, ...",misc,"['1427713523817', '65439a62c552e8efa629c098edb..."
3,Whether the ‘sponsor’ and ‘originator’ of a Sp...,It is to be noted that there is no specific pr...,9f7c59b16f91b9399a9ac477968a0dcb,"Sponsor, Originator, Special Purpose distinct ...",misc,"['1cd583e20cd37da37530d0131f217618', '9f7c59b1..."
4,What are the effects and consequences of not r...,I find that the details sought by the IA were ...,1289453383303,"Noticee, Summons, failure to Comply with summo...",legal cases,"['1529063091218', '1604401204480_3', '15452021..."
5,Who is an ‘acquirer’ under SAST and which prov...,"In terms of regulation 7(1) of SAST, the oblig...",1290154724736_NEG,"Acquirer, SAST,",legal cases,"['1533038755477', '1516011484965', '1529063091..."
6,How is the term ‘acquisition’ under PIT Regula...,"As regards regulation 13 (1) of PIT, the discl...",1290154724736_NEG,"acquistion, Acquirer",legal cases,"['1533038755477', '1516011484965', '1529063091..."
7,What are the ‘salient features’ of the SEBI ( ...,The salient features of these guidelines are b...,1292328051155,"Delisting guidelines, features",misc,"['1450433911091', '1453278327759', '1366279014..."
8,What is the mission of SEBI?,The mission of SEBI is to make India as one of...,1292831218021,"SEBI, Mission",misc,"['1450436313801', '1487248534562', '1453278327..."
9,What is Market Participants’ and Investors’ Id...,The Central Database of Securities Market Part...,1292909401123,Market Participants’ and Investors’ Identifica...,misc,"['1450436313801', '1415184470668', '1448966221..."


In [4]:
df[["Answer Span",df.columns[2]]]

,Answer Span,Document
0,"With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e
1,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5
2,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639
3,It is to be noted that there is no specific pr...,9f7c59b16f91b9399a9ac477968a0dcb
4,I find that the details sought by the IA were ...,1289453383303
5,"In terms of regulation 7(1) of SAST, the oblig...",1290154724736_NEG
6,"As regards regulation 13 (1) of PIT, the discl...",1290154724736_NEG
7,The salient features of these guidelines are b...,1292328051155
8,The mission of SEBI is to make India as one of...,1292831218021
9,The Central Database of Securities Market Part...,1292909401123


In [ ]:
starter = [
    "With respect to the query at",
    "In the instant case,",
    "The provisions under Regulation",
    
    
    
]

In [25]:
def fileType(name):
    if name+".txt" in lfile:
        return "legal cases"
    if name in docs:
        return "regulations"
    return "misc"
df["File Type"]=df[df.columns[2]].apply(fileType)
df

,Question,Answer Span,Document,Keywords,File Type
0,Whether any entity that qualifies as a promote...,"With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e,"pre-clearance, compliance officer, designated ...",misc
1,Whether inter-se off-market transfer of shares...,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5,"inter-se off market transfer, block deal, cont...",misc
2,Whether an Alternative Investment Fund under t...,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639,"unutilized funds, Alternate Investment Funds, ...",misc
3,Whether the ‘sponsor’ and ‘originator’ of a Sp...,It is to be noted that there is no specific pr...,9f7c59b16f91b9399a9ac477968a0dcb,"Sponsor, Originator, Special Purpose distinct ...",misc
4,What are the effects and consequences of not r...,I find that the details sought by the IA were ...,1289453383303,"Noticee, Summons, failure to Comply with summo...",legal cases
5,Who is an ‘acquirer’ under SAST and which prov...,"In terms of regulation 7(1) of SAST, the oblig...",1290154724736_NEG,"Acquirer, SAST,",legal cases
6,How is the term ‘acquisition’ under PIT Regula...,"As regards regulation 13 (1) of PIT, the discl...",1290154724736_NEG,"acquistion, Acquirer",legal cases
7,What are the ‘salient features’ of the SEBI ( ...,The salient features of these guidelines are b...,1292328051155,"Delisting guidelines, features",misc
8,What is the mission of SEBI?,The mission of SEBI is to make India as one of...,1292831218021,"SEBI, Mission",misc
9,What is Market Participants’ and Investors’ Id...,The Central Database of Securities Market Part...,1292909401123,Market Participants’ and Investors’ Identifica...,misc


In [65]:
def keywordsProcessing(words,filetype):
    scorer=dict()
    words=words.split(",")
    words = [ w.strip() for w in words]
    if filetype== "legal cases":
        for k,v in caseMap.items():
            count = 0
            for w in words:
                if w.lower() in k.lower():
                    count+=1
            if v in scorer.keys() and scorer[v] < count :
                scorer[v]=count
            if v not in scorer.keys():
                scorer[v]=count
            
    elif filetype== "regulations":
        for k,v in regMap.items():
            count = 0
            for w in words:
                if w.lower() in k.lower():
                    count+=1
            if v in scorer.keys() and scorer[v] < count :
                scorer[v]=count
            if v not in scorer.keys():
                scorer[v]=count 
                
    else:
        for k,v in miscMap.items():
            count = 0
            for w in words:
                if w.lower() in k.lower():
                    count+=1
            if v in scorer.keys() and scorer[v] < count :
                scorer[v]=count
            if v not in scorer.keys():
                scorer[v]=count
    
    filename= [k for k, v in sorted(scorer.items(), key=lambda item: item[1],reverse=True)]
    
    return filename[:10]

df["Files For Keywords"]=df.apply(lambda x: keywordsProcessing(x["Keywords"],x["File Type"]),axis=1)
df

,Question,Answer Span,Document,Keywords,File Type,Files For Keywords
0,Whether any entity that qualifies as a promote...,"With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e,"pre-clearance, compliance officer, designated ...",misc,"[7db3e9ce43373e8d2823b7603b72252b, 853914f4f77..."
1,Whether inter-se off-market transfer of shares...,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5,"inter-se off market transfer, block deal, cont...",misc,"[08137e72ee44f3d2f24e42b2ca270de6, 830be2cfb98..."
2,Whether an Alternative Investment Fund under t...,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639,"unutilized funds, Alternate Investment Funds, ...",misc,"[1427713523817, 65439a62c552e8efa629c098edbe2c..."
3,Whether the ‘sponsor’ and ‘originator’ of a Sp...,It is to be noted that there is no specific pr...,9f7c59b16f91b9399a9ac477968a0dcb,"Sponsor, Originator, Special Purpose distinct ...",misc,"[1cd583e20cd37da37530d0131f217618, 9f7c59b16f9..."
4,What are the effects and consequences of not r...,I find that the details sought by the IA were ...,1289453383303,"Noticee, Summons, failure to Comply with summo...",legal cases,"[1529063091218, 1604401204480_3, 1545202196705..."
5,Who is an ‘acquirer’ under SAST and which prov...,"In terms of regulation 7(1) of SAST, the oblig...",1290154724736_NEG,"Acquirer, SAST,",legal cases,"[1533038755477, 1516011484965, 1529063091218, ..."
6,How is the term ‘acquisition’ under PIT Regula...,"As regards regulation 13 (1) of PIT, the discl...",1290154724736_NEG,"acquistion, Acquirer",legal cases,"[1533038755477, 1516011484965, 1529063091218, ..."
7,What are the ‘salient features’ of the SEBI ( ...,The salient features of these guidelines are b...,1292328051155,"Delisting guidelines, features",misc,"[1450433911091, 1453278327759, 1366279014828, ..."
8,What is the mission of SEBI?,The mission of SEBI is to make India as one of...,1292831218021,"SEBI, Mission",misc,"[1450436313801, 1487248534562, 1453278327759, ..."
9,What is Market Participants’ and Investors’ Id...,The Central Database of Securities Market Part...,1292909401123,Market Participants’ and Investors’ Identifica...,misc,"[1450436313801, 1415184470668, 1448966221407, ..."


In [68]:
df.to_csv("QnA.csv",index=False)

In [88]:
def contextFinder(x,y):
    if y=="misc":
        rules = cfile
        rules.extend(ifile)
        sentences = csent
        sentences.extend(isent)
        try:
            return sentences[rules.index(x+".docx")]
        except:
            return sentences[rules.index(x+".txt")]
            
    elif y=="legal cases":
        return lsent[lfile.index(x+".txt")]
    elif y=="regulations":
        return docregs[docs.index(x)] ## check if this is correct
    return ""

document=df.columns[2]
df["context"]=df.apply(lambda x: contextFinder(x[document],x["File Type"]),axis=1)
df

,Question,Answer Span,Document,Keywords,File Type,context
0,Whether any entity that qualifies as a promote...,"With respect to the query at 5(i), attention m...",0c9f4131313e54e49d7b9915993ab00e,"pre-clearance, compliance officer, designated ...",misc,"[griff.If7, \WITr: 1 if ti ci, Securities and ..."
1,Whether inter-se off-market transfer of shares...,"In the instant case, the said promoters have t...",0e8e621eb1aee80e102b4186ac874ed5,"inter-se off market transfer, block deal, cont...",misc,[ Request for Informal Guidance under Securiti...
2,Whether an Alternative Investment Fund under t...,The provisions under Regulation 15(1)(f) is pr...,1e4e946eca55a80467aad2e1546dd639,"unutilized funds, Alternate Investment Funds, ...",misc,"[g1' ( alzre1 Tfa-, 34frrcirviHq, Securities a..."
3,Whether the ‘sponsor’ and ‘originator’ of a Sp...,It is to be noted that there is no specific pr...,9f7c59b16f91b9399a9ac477968a0dcb,"Sponsor, Originator, Special Purpose distinct ...",misc,[ Informal guidance by way of an interpretive ...
4,What are the effects and consequences of not r...,I find that the details sought by the IA were ...,1289453383303,"Noticee, Summons, failure to Comply with summo...",legal cases,[ - Adjudication%200Ramswaro… BEFORE THE ADJUD...
5,Who is an ‘acquirer’ under SAST and which prov...,"In terms of regulation 7(1) of SAST, the oblig...",1290154724736_NEG,"Acquirer, SAST,",legal cases,[ - … BEFORE THE ADJUDICATING OFFICER SECURITI...
6,How is the term ‘acquisition’ under PIT Regula...,"As regards regulation 13 (1) of PIT, the discl...",1290154724736_NEG,"acquistion, Acquirer",legal cases,[ - … BEFORE THE ADJUDICATING OFFICER SECURITI...
7,What are the ‘salient features’ of the SEBI ( ...,The salient features of these guidelines are b...,1292328051155,"Delisting guidelines, features",misc,[C o n c e p t   P a p e r   o n   t h e   p r...
8,What is the mission of SEBI?,The mission of SEBI is to make India as one of...,1292831218021,"SEBI, Mission",misc,[S E C U R I T I E S   A N D   E X C H A N G E...
9,What is Market Participants’ and Investors’ Id...,The Central Database of Securities Market Part...,1292909401123,Market Participants’ and Investors’ Identifica...,misc,[R E P O R T   O F   T H E   S E B I   C O M M...


In [109]:
temp = df[df["File Type"]=="regulations"]
temp.reset_index()
for i,d in temp.iterrows():
    for c in d["context"]:
        flag=False
        if d["Answer Span"] in c:
            print(c)
            flag=False
            break
    if not flag:
        print(i)
            
df["Answer Span"][1]

25
26
27
28
29
30
31
32
33
34


'In the instant case, the said promoters have the option to convert warrants any time within 18 months from the date of allotment in one or more tranches. The subsequent sale within 6 months may attract the contra-trade restrictions under the PIT Regulations. Likewise, if the promoters or members of the promoter group who had acquired shares through inter-se off market transfer of shares or through block deal window mechanism between promoters or members of the promoter group want to transfer shares to the acquirer trusts within 6 months, the proposed transfer to the acquirer trust within 6 months will also attract the contra-trade restrictions specified under the PIT Regulations.'

In [126]:
# temp.reset_index()
# temp["Answer Span"][26]

In [128]:
df["context"][3]

[' Informal guidance by way of an interpretive letter under the SEBI ( Informal Guidance ) Scheme, 2003',
 'This has reference to your letter dated October 14, 2019 wherein you have sought informal guidance by way of an "Interpretive letter" from SEBI under paragraph 5 ( ii ) of the SEBI ( Informal Guidance ) Scheme, 2003.',
 'In your letter under reference, you have, inter-a/ia, represented the following:',
 'Beacon Trusteeship Limited ( the "Trustee" or "Beacon" ) is registered with the SEBI as a debenture trustee since April 11, 2016 bearing registration number IND000000569. Beacon was incorporated on December 23, 2015 with the main object of providing trusteeship services by a group of ex-bankers and professionals with vast experience in the trusteeship business.',
 'Trust Investment Advisors Private Limited ( the "Sponsor" and "Originator" ) is registered with SEBI as a Category 1 Merchant Banker since 29.03.2007 bearing registration number INM00001 1120 and as portfolio Manager s

In [68]:
rules = cfile
rules.extend(ifile)
sentences = csent
sentences.extend(isent)
rules.index("0e8e621eb1aee80e102b4186ac874ed5.docx")

194

In [23]:
# sentences[210]
# ifile
from os import listdir
from os.path import isfile, join
mypath = "PDFS/"
count=0
misc_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# mypath = "Informal Guidance/"
# ifile = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for i in ifile:
    if i in misc_files:
        count+=1
    else:
        print(i)
        
print(count)

385


In [22]:
len(ifile)

385

In [124]:
# lsent[lfile.index("1290154724736_NEG"+".txt")]

In [123]:
# df["context"][5]

In [122]:
# lfile

In [121]:
# ctext

In [120]:
# itext